# Table of contents
1. [Introduction](#introduction)
1. [Install pre-release of CLTK](#install)
1. [Get data](#get-data)
1. [Run NLP pipeline with `NLP()`](#run-nlp)
1. [Inspect CLTK `Doc`](#inspect-doc)
1. [Inspect CLTK `Word`](#inspect-word)
1. [Modeling morphology with `MorphosyntacticFeature` and `MorphosyntacticFeatureBundle`](#morph)
1. [Modeling syntax with `Form` and `DependencyTree`](#syntax)
1. [Feature extraction](#features)
1. [Brief demonstration of `NLP()` for Ancient Greek](#greek-nlp)

# Introduction <a name="introduction"></a>

This notebook demonstrates how to use `NLP()`, the CLTK's primary interface, in Latin and Ancient Greek. Pipelines are available for 17 languages (see [Languages](https://docs.cltk.org/en/latest/languages.html) in the docs).

Full documentation available at <https://docs.cltk.org/en/latest/cltk.html#cltk.nlp.NLP>.

Note that there is a large amoung of code from this project's first six years (v. `0.1`), not all of which has been or will be moved over to this v. `1.0`. Docs for `0.1` still available at [https://legacy.cltk.org](https://legacy.cltk.org/) and tutorial notebooks at [https://github.com/cltk/tutorials](https://github.com/cltk/tutorials).

# Install CLTK <a name="install"></a>

This notebook comes from <https://github.com/cltk/cltk/tree/master/notebooks>. For full instructions on installing the CLTK are available at <https://docs.cltk.org/en/latest/installation.html>.

In [1]:
# !pip install cltk

In [2]:
## Or directly from this repo:
# cd .. && make install

%load_ext autoreload
%autoreload 2

# Get data <a name="get-data"></a>

The following obtain two plaintext documents of two Classical authors. A subset of each will be used to demonstrate the CLTK.

In [3]:
# Get Latin text
# https://gist.github.com/kylepjohnson/2f9376fcf15699c250a0d09b37683370
# now at `notebooks/lat-livy.txt`
# !curl -O -L https://gist.github.com/kylepjohnson/2f9376fcf15699c250a0d09b37683370/raw/05b7a17af4b216a4986d897c57a9987e836cc91a/lat-livy.txt

In [4]:
# Get Ancient Greek text
# https://gist.github.com/kylepjohnson/9835c36fb06ca30ebf29b7f2c7bd29e0
# now at `notebooks/grc-thucydides.txt`
# !curl -O -L https://gist.github.com/kylepjohnson/9835c36fb06ca30ebf29b7f2c7bd29e0/raw/e08e47849f64484b0950b14563bb5a9fd1e1ef1c/grc-thucydides.txt

In [5]:
# read the Latin file
# We'll run the full demonstration in the Latin language first
with open("lat-livy.txt") as fo:
    livy_full = fo.read()

In [6]:
print("Text snippet:", livy_full[:200])
print("Character count:", len(livy_full))
print("Approximate token count:", len(livy_full.split()))

Text snippet: Iam primum omnium satis constat Troia capta in ceteros saevitum esse Troianos, duobus, Aeneae Antenorique, et vetusti iure hospitii et quia pacis reddendaeque Helenae semper auctores fuerant, omne ius
Character count: 920884
Approximate token count: 129799


In [7]:
len(livy_full) // 12

76740

In [8]:
# Now let's cut this down to roughly 10k tokens for this demonstration's purposes
livy = livy_full[:len(livy_full) // 12]
print("Approximate token count:", len(livy.split()))

Approximate token count: 10911


# Run NLP pipeline with `NLP()` <a name="run-nlp"></a>

In [9]:
# For most users, this is the only import required
from cltk import NLP

In [10]:
# Load the default Pipeline for Latin
cltk_nlp = NLP(language="lat")

‎𐤀 CLTK version '1.2.0'. When using the CLTK in research, please cite: https://aclanthology.org/2021.acl-demo.3/

Pipeline for language 'Latin' (ISO: 'lat'): `LatinNormalizeProcess`, `LatinSpacyProcess`, `LatinEmbeddingsProcess`, `StopsProcess`, `LatinLexiconProcess`.

⸖ ``LatinSpacyProcess`` using LatinCy model by Patrick Burns from https://arxiv.org/abs/2305.04365 . Please cite: https://arxiv.org/abs/2305.04365
⸖ ``LatinEmbeddingsProcess`` using word2vec model by University of Oslo from http://vectors.nlpl.eu/ . Please cite: https://aclanthology.org/W17-0237/
⸖ ``LatinLexiconProcess`` using Lewis's *An Elementary Latin Dictionary* (1890).

⸎ To suppress these messages, instantiate ``NLP()`` with ``suppress_banner=True``.


In [11]:
# Removing ``LatinLexiconProcess`` for this demo b/c it is slow (adds ~9 mins total)
cltk_nlp.pipeline.processes.pop(-1)
print(cltk_nlp.pipeline.processes)

[<class 'cltk.alphabet.processes.LatinNormalizeProcess'>, <class 'cltk.dependency.processes.LatinSpacyProcess'>, <class 'cltk.embeddings.processes.LatinEmbeddingsProcess'>, <class 'cltk.stops.processes.StopsProcess'>]


In [12]:
# Now execute NLP algorithms upon input text
# Aside from download, execution time is ~30 sec on a 2015 Macbook Pro
%time cltk_doc = cltk_nlp.analyze(text=livy)

# You will be asked to download some models (from CLTK, fastText, and Stanza)

CPU times: user 30.5 s, sys: 1.24 s, total: 31.7 s
Wall time: 31.9 s


# Inspect CLTK `Doc` <a name="inspect-doc"></a>

In [13]:
# We can now inspect the result
print(type(cltk_doc))

<class 'cltk.core.data_types.Doc'>


In [14]:
# All accessors
print([x for x in dir(cltk_doc) if not x.startswith("__")])

['_get_words_attribute', 'embeddings', 'embeddings_model', 'language', 'lemmata', 'morphosyntactic_features', 'normalized_text', 'pipeline', 'pos', 'raw', 'sentence_embeddings', 'sentences', 'sentences_strings', 'sentences_tokens', 'spacy_doc', 'stems', 'tokens', 'tokens_stops_filtered', 'words']


In [15]:
# Several of the more useful

# List of tokens
print(cltk_doc.tokens[:20])

['Iam', 'primum', 'omnium', 'satis', 'constat', 'Troia', 'capta', 'in', 'ceteros', 'saevitum', 'esse', 'Troianos', ',', 'duobus', ',', 'Aeneae', 'Antenori', 'que', ',', 'et']


In [16]:
# List of lemmas
print(cltk_doc.lemmata[:20])

['iam', 'primus', 'omnis', 'satisfacio', 'consto', 'Troia', 'capio', 'in', 'ceterus', 'saevitum', 'sum', 'Troiani', ',', 'duo', ',', 'Aeneae', 'Antenori', 'que', ',', 'et']


In [17]:
# Basic part-of-speech info
print(cltk_doc.pos[:20])

['ADV', 'ADJ', 'ADJ', 'ADV', 'VERB', 'PROPN', 'VERB', 'ADP', 'DET', 'VERB', 'AUX', 'ADJ', 'PUNCT', 'NUM', 'PUNCT', 'PROPN', 'PROPN', 'NOUN', 'PUNCT', 'CCONJ']


In [18]:
# A list of list of tokens
print(cltk_doc.sentences_tokens[:2])

[['Iam', 'primum', 'omnium', 'satis', 'constat', 'Troia', 'capta', 'in', 'ceteros', 'saevitum', 'esse', 'Troianos', ',', 'duobus', ',', 'Aeneae', 'Antenori', 'que', ',', 'et', 'vetusti', 'iure', 'hospitii', 'et', 'quia', 'pacis', 'reddendae', 'que', 'Helenae', 'semper', 'auctores', 'fuerant', ',', 'omne', 'ius', 'belli', 'Achiuos', 'abstinuisse', ';'], ['casibus', 'deinde', 'variis', 'Antenorem', 'cum', 'multitudine', 'Enetum', ',', 'qui', 'seditione', 'ex', 'Paphlagonia', 'pulsi', 'et', 'sedes', 'et', 'ducem', 'rege', 'Pylaemene', 'ad', 'Troiam', 'amisso', 'quaerebant', ',', 'venisse', 'in', 'intimum', 'maris', 'Hadriatici', 'sinum', ',', 'Euganeis', 'que', 'qui', 'inter', 'mare', 'Alpes', 'que', 'incolebant', 'pulsis', 'Enetos', 'Troianos', 'que', 'eas', 'tenuisse', 'terras', '.']]


# Inspect CLTK `Word` <a name="inspect-word"></a>

Most powerful, though, is the ``Doc.words`` accessor, which is a list of ``Word`` objects. These ``Word`` objects contain all information that was generated during the NLP pipeline

In [19]:
# One ``Word`` object for each token
print(len(cltk_doc.words))

12986


Users can go token-by-token via ``Doc.words`` or via the intermediary step of looping through sentences.

In [20]:
# Let's look at a non-trivial sentence from Book 1
print("Original:", cltk_doc.sentences_strings[5])
print("")
print("Translation:", "Landing there, the Trojans, as men who, after their all but immeasurable wanderings, had nothing left but their swords and ships, were driving booty from the fields, when King Latinus and the Aborigines, who then occupied that country, rushed down from their city and their fields to repel with arms the violence of the invaders.")
# source: http://www.perseus.tufts.edu/hopper/text?doc=Liv.+1+1+5&fromdoc=Perseus%3Atext%3A1999.02.0151
sentence_6 = cltk_doc.sentences[5]  # type: list[Word]

Original: Ibi egressi Troiani , ut quibus ab immenso prope errore nihil praeter arma et naues superesset , cum praedam ex agris agerent , Latinus rex Aborigines que qui tum ea tenebant loca ad arcendam vim advenarum armati ex urbe atque agris concurrunt .

Translation: Landing there, the Trojans, as men who, after their all but immeasurable wanderings, had nothing left but their swords and ships, were driving booty from the fields, when King Latinus and the Aborigines, who then occupied that country, rushed down from their city and their fields to repel with arms the violence of the invaders.


In [21]:
# Looking at one Word, 'concurrunt' ('they run together')
a_word_concurrunt = sentence_6[41]
print(a_word_concurrunt)

Word(index_char_start=1091, index_char_stop=1101, index_token=189, index_sentence=5, string='concurrunt', pos=verb, lemma='concurro', stem=None, scansion=None, xpos='verb', upos='VERB', dependency_relation='ROOT', governor=189, features={Mood: [indicative], Number: [plural], Person: [third], Tense: [present], VerbForm: [finite], Voice: [active]}, category={F: [neg], N: [neg], V: [pos]}, stop=False, named_entity=None, syllables=None, phonetic_transcription=None, definition=None)


In this word, you can see information for lexicography (`.lemmata`), semantics (`.embedding`), morphology (`.pos`, `.features`), syntax (`.governor`, `.dependency_relation`), plus other information most users would find helpful (`.stop`, `.named_entity`).

# Modeling morphology with `MorphosyntacticFeature` and `MorphosyntacticFeatureBundle` <a name="morph"></a>

When a language's `Pipeline` builds each `Word` object, morphological information is stored at several accessors. Those of interest to most users are `.pos` and `.features`.

In [22]:
print("`Word.string`:", a_word_concurrunt.string)
print("")
# Part-of-speech is always be available at `.pos`.
print("`Word.pos`:", a_word_concurrunt.pos)

`Word.string`: concurrunt

`Word.pos`: verb


The CLTK contains classes a specific class for [the annotation types defined by v2 of the Universal Dependencies project](https://universaldependencies.org/u/feat/all.html). In the CLTK's codebase, these are located at [cltk/morphology/universal_dependencies_features.py](https://github.com/cltk/cltk/blob/dev/src/cltk/morphology/universal_dependencies_features.py).

For instance, a Latin verb requires a label for its [https://universaldependencies.org/u/feat/all.html#al-u-feat/Mood](mood) (e.g., indicative), which the UD project defines as "a feature that expresses modality and subclassifies finite verb forms".

Though morphological taggers may annnotate a verb's mood variously ("ind.", "indicative", "Indic", etc.), the CLTK maps the term into the following, standardized `Mood`.

``` python
class Mood(MorphosyntacticFeature):
    """The mood of a verb.
    see https://universaldependencies.org/u/feat/Mood.html
    """

    admirative = auto()
    conditional = auto()
    desiderative = auto()
    imperative = auto()
    indicative = auto()
    jussive = auto()
    necessitative = auto()
    optative = auto()
    potential = auto()
    purposive = auto()
    quotative = auto()
    subjunctive = auto()
```

Turning back to the the above example word, we can see such features at `.features`.

In [23]:
# type
print("type(`Word.features`):", type(a_word_concurrunt.features))
print("")
# str repr of `MorphosyntacticFeatureBundle`
print("`Word.features`:", a_word_concurrunt.features)

type(`Word.features`): <class 'cltk.morphology.morphosyntax.MorphosyntacticFeatureBundle'>

`Word.features`: {Mood: [indicative], Number: [plural], Person: [third], Tense: [present], VerbForm: [finite], Voice: [active]}


A user may inspect a `MorphosyntacticFeatureBundle` in a manner similar to a `dict`

In [24]:
print("Mood:", a_word_concurrunt.features["Mood"])  # type: list[Mood]
print("Number:", a_word_concurrunt.features["Number"])  # type: list[Number]
print("Person:", a_word_concurrunt.features["Person"])  # type: list[Person]
print("Tense:", a_word_concurrunt.features["Tense"])  # type: list[Tense]
print("VerbForm:", a_word_concurrunt.features["VerbForm"])  # type: list[VerbForm]
print("Voice:", a_word_concurrunt.features["Voice"])  # type: list[Voice]

# Note: The values returned here are a list, though under normally only one 
# morphological form will be available

Mood: [indicative]
Number: [plural]
Person: [third]
Tense: [present]
VerbForm: [finite]
Voice: [active]


Looking a bit closer at `MorphosyntacticFeature`, we can see how its data type inherits from the Python builtin [IntEnu](https://docs.python.org/3/library/enum.html#enum.IntEnum).

In [25]:
a_mood_obj = a_word_concurrunt.features["Mood"][0]
# see type
print("type(a_mood_obj):", type(a_mood_obj))
print("")
# See inheritance
from enum import IntEnum
print("Is `IntEnum`?", isinstance(a_mood_obj, IntEnum))
print("")
# 
from cltk.morphology.morphosyntax import MorphosyntacticFeature
print("`Mood` inherits from `MorphosyntacticFeature`?", isinstance(a_mood_obj, MorphosyntacticFeature))

type(a_mood_obj): Mood

Is `IntEnum`? True

`Mood` inherits from `MorphosyntacticFeature`? True


In [26]:
# You can manipulate this object as any IntEnum plus a few extras

print("`MorphosyntacticFeature` accessors:", [x for x in dir(a_mood_obj) if not x.startswith("__")])
print("")
print("MorphosyntacticFeature.name:", a_mood_obj.name)  # type: str
# A stable int value is available, too, associated with this name
print("MorphosyntacticFeature.value:", a_mood_obj.value)  # type: int

`MorphosyntacticFeature` accessors: ['_generate_next_value_', '_member_map_', '_member_names_', '_member_type_', '_missing_', '_name_', '_new_member_', '_sort_order_', '_unhashable_values_', '_use_args_', '_value2member_map_', '_value_', '_value_repr_', 'admirative', 'as_integer_ratio', 'bit_count', 'bit_length', 'conditional', 'conjugate', 'denominator', 'desiderative', 'from_bytes', 'imag', 'imperative', 'indicative', 'jussive', 'name', 'necessitative', 'numerator', 'optative', 'potential', 'purposive', 'quotative', 'real', 'subjunctive', 'to_bytes', 'value']

MorphosyntacticFeature.name: indicative
MorphosyntacticFeature.value: 5


Users can create their own `MorphosyntacticFeature` and `MorphosyntacticFeatureBundle`:

In [27]:
from cltk.morphology.morphosyntax import MorphosyntacticFeatureBundle
from cltk.morphology.universal_dependencies_features import Mood, Number, Person, VerbForm, Voice

latin_word_sim = "sim"

mood = Mood.subjunctive
voice = Voice.active
person = Person.first
number = Number.singular
verb_form = VerbForm.finite

latin_word_sim_bundle = MorphosyntacticFeatureBundle(mood, voice, person, number, verb_form)
print(latin_word_sim_bundle)

{Mood: [subjunctive], Voice: [active], Person: [first], Number: [singular], VerbForm: [finite]}


This too can be interated through as `dict`:

In [28]:
for feature, value in latin_word_sim_bundle.items():
    print(feature, value)  # Note: value has `list` type

Mood [subjunctive]
Voice [active]
Person [first]
Number [singular]
VerbForm [finite]


Finally, we may even construct a `Word` with this information:

In [29]:
from cltk.core.data_types import Word

print(Word(string="sim", features=latin_word_sim_bundle))

Word(index_char_start=None, index_char_stop=None, index_token=None, index_sentence=None, string='sim', pos=None, lemma=None, stem=None, scansion=None, xpos=None, upos=None, dependency_relation=None, governor=None, features={Mood: [subjunctive], Voice: [active], Person: [first], Number: [singular], VerbForm: [finite]}, category={}, stop=None, named_entity=None, syllables=None, phonetic_transcription=None, definition=None)


In [30]:
# For more on this or any other CLTK class, use `help()`
# help(a_mood_obj)
# help(MorphosyntacticFeatureBundle)

In [31]:
# Note: Extra morphological info may be written in `str` type
# to to the values at `.upos` and `.xpos` for languages using
# Stanza project

# Note: The particular annoations at these are often inconsistent across
# languages or even treebanks within a single language; hence the benefit
# of the CLTK's modeling at `.pos`.
print("`Word.upos`:", a_word_concurrunt.upos)
print("`Word.xpos`:", a_word_concurrunt.xpos)

`Word.upos`: VERB
`Word.xpos`: verb


# Modeling syntax with `Form` and `DependencyTree`  <a name="syntax"></a>

The CLTK uses the builtin `xml` library to build tree for modeling dependency parses. A `Word` is mapped into a `Form`, then `ElemntTree` is used to organize these `Form`s into a `DependencyTree`. With a tree, certain measurements are more efficient (counting depth, breadth, edge types).

In [32]:
from cltk.dependency.tree import DependencyTree

In [33]:
# Let's look at this sentence again
print(cltk_doc.sentences_strings[5])  # text form of `sentence_6`

Ibi egressi Troiani , ut quibus ab immenso prope errore nihil praeter arma et naues superesset , cum praedam ex agris agerent , Latinus rex Aborigines que qui tum ea tenebant loca ad arcendam vim advenarum armati ex urbe atque agris concurrunt .


In [34]:
a_tree = DependencyTree.to_tree(sentence_6)

In [35]:
a_tree

In [36]:
from pprint import pprint

pprint(a_tree.get_dependencies())

[advmod(concurrunt_189/verb, Ibi_148/adverb),
 advcl(concurrunt_189/verb, egressi_149/verb),
 advcl(concurrunt_189/verb, armati_184/verb),
 punct(Troiani_150/noun, ,_151/punctuation),
 mark(superesset_163/verb, ut_152/subordinating_conjunction),
 obl:arg(superesset_163/verb, quibus_153/pronoun),
 case(immenso_155/adjective, ab_154/adposition),
 obl(superesset_163/verb, immenso_155/adjective),
 obl(superesset_163/verb, errore_157/noun),
 obl(superesset_163/verb, arma_160/noun),
 case(errore_157/noun, prope_156/adposition),
 obl(superesset_163/verb, immenso_155/adjective),
 obl(superesset_163/verb, errore_157/noun),
 obl(superesset_163/verb, arma_160/noun),
 obj(superesset_163/verb, nihil_158/noun),
 case(arma_160/noun, praeter_159/adposition),
 cc(naues_162/noun, et_161/coordinating_conjunction),
 conj(arma_160/noun, naues_162/noun),
 obl(superesset_163/verb, immenso_155/adjective),
 obl(superesset_163/verb, errore_157/noun),
 obl(superesset_163/verb, arma_160/noun),
 punct(superesset_1

In [37]:
a_tree.print_tree()

root | concurrunt_189/verb
    └─ advmod | Ibi_148/adverb
    └─ advcl | egressi_149/verb
    └─ nsubj | Troiani_150/noun
        └─ punct | ,_151/punctuation
        └─ appos | rex_172/noun
            └─ advcl | superesset_163/verb
                └─ mark | ut_152/subordinating_conjunction
                └─ obl:arg | quibus_153/pronoun
                └─ obl | immenso_155/adjective
                    └─ case | ab_154/adposition
                └─ obl | errore_157/noun
                    └─ case | prope_156/adposition
                └─ obj | nihil_158/noun
                └─ obl | arma_160/noun
                    └─ case | praeter_159/adposition
                    └─ conj | naues_162/noun
                        └─ cc | et_161/coordinating_conjunction
                └─ punct | ,_164/punctuation
            └─ advcl | agerent_169/verb
                └─ mark | cum_165/subordinating_conjunction
                └─ obj | praedam_166/noun
                └─ obl | agris_168/noun
    

# Feature extraction <a name="features"></a>

The CLTK offers the function `cltk_doc_to_features_table()`, which assist users when preparing a `Doc` for training data for machine learning. It converts the list of `Word` objects at `Doc.words` into a tabular list of lists.

In [38]:
from cltk.utils.feature_extraction import cltk_doc_to_features_table

In [39]:
feature_names, list_of_list_features = cltk_doc_to_features_table(cltk_doc=cltk_doc)

In [40]:
# See here the names of the features extracted
print(feature_names)

['string', 'lemma', 'embedding', 'is_stop', 'lemma', 'pos', 'case', 'gender', 'animacy', 'number', 'definiteness', 'degree', 'strength', 'verbform', 'tense', 'mood', 'aspect', 'voice', 'person', 'polarity', 'politeness', 'clusivity', 'evidentiality', 'strength', 'governing_word', 'governor_case', 'governor_gender', 'governor_animacy', 'governor_number', 'governor_definiteness', 'governor_degree', 'governor_strength', 'governor_verbform', 'governor_tense', 'governor_mood', 'governor_aspect', 'governor_voice', 'governor_person', 'governor_polarity', 'governor_politeness', 'governor_clusivity', 'governor_evidentiality', 'governor_strength', 'governing_relationship']


In [41]:
# Number of "inner lists" matches number of tokens
print("Number tokens:", len(cltk_doc.words))
print("Number of feature instances (one for each token):", len(list_of_list_features))

Number tokens: 12986
Number of feature instances (one for each token): 12986


In [42]:
# Look at one row of data `(variable name, variable value)`
pprint(list(zip(feature_names, list_of_list_features[108])))

[('string', 'simili'),
 ('lemma', 'similis'),
 ('embedding',
  array([ 0.010836 , -0.12199  ,  0.39812  , -0.15676  , -0.10664  ,
       -0.1718   , -0.50263  , -0.24026  ,  0.31116  ,  0.65852  ,
       -0.086368 ,  0.33481  ,  0.18166  , -0.3847   ,  0.19406  ,
       -0.11596  ,  0.47762  , -0.15744  ,  0.40811  , -0.25476  ,
        0.68516  ,  0.23404  ,  0.35565  , -0.21808  , -0.19027  ,
       -0.1504   ,  0.47524  , -0.19055  ,  0.25193  ,  0.19534  ,
       -0.36006  ,  0.39979  ,  0.3842   ,  0.40202  ,  0.10175  ,
        0.037091 ,  0.10014  , -0.19205  , -0.12977  , -0.41553  ,
        0.6148   ,  0.027429 ,  0.31857  , -0.18587  ,  0.20306  ,
       -0.12926  ,  0.24522  , -0.029757 ,  0.53599  ,  0.098517 ,
       -0.24308  , -0.29966  , -0.46403  ,  0.041967 ,  0.48501  ,
       -0.043341 ,  0.39231  ,  0.50658  , -0.087431 , -0.52358  ,
       -0.59075  , -0.30634  ,  0.19888  ,  0.12592  , -0.44104  ,
       -0.056771 , -0.81705  , -0.012789 , -0.38883  , -0.56507  ,

# Brief demonstration of `NLP()` for Ancient Greek <a name="greek-nlp"></a>

The API for Greek is the same as Latin.

In [43]:
# read the Ancient Greek file
with open("grc-thucydides.txt") as fo:
    thucydides_full = fo.read()

In [44]:
print("Text snippet:", thucydides_full[0:200])
print("Character count:", len(thucydides_full))
print("Approximate token count:", len(thucydides_full.split()))

Text snippet: Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν Πελοποννησίων καὶ Ἀθηναίων, ὡς ἐπολέμησαν πρὸς ἀλλήλους, ἀρξάμενος εὐθὺς καθισταμένου καὶ ἐλπίσας μέγαν τε ἔσεσθαι καὶ ἀξιολογώτατον τῶν προγεγενημένων, τ
Character count: 470056
Approximate token count: 72619


In [45]:
len(thucydides_full) // 7

67150

In [46]:
# Cut this down to roughly 10k tokens for this demonstration's purposes
thucydides = thucydides_full[:len(thucydides_full) // 7]
print("Approximate token count:", len(thucydides.split()))

Approximate token count: 10309


In [47]:
thucydides[:200]

'Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν Πελοποννησίων καὶ Ἀθηναίων, ὡς ἐπολέμησαν πρὸς ἀλλήλους, ἀρξάμενος εὐθὺς καθισταμένου καὶ ἐλπίσας μέγαν τε ἔσεσθαι καὶ ἀξιολογώτατον τῶν προγεγενημένων, τ'

In [48]:
cltk_nlp_grc = NLP(language="grc")

‎𐤀 CLTK version '1.2.0'. When using the CLTK in research, please cite: https://aclanthology.org/2021.acl-demo.3/

Pipeline for language 'Ancient Greek' (ISO: 'grc'): `GreekNormalizeProcess`, `GreekStanzaProcess`, `GreekEmbeddingsProcess`, `StopsProcess`.

⸖ ``LatinSpacyProcess`` using Stanza model by Stanford University from https://stanfordnlp.github.io/stanza/ . Please cite: https://arxiv.org/abs/2003.07082
⸖ ``LatinEmbeddingsProcess`` using word2vec model by University of Oslo from http://vectors.nlpl.eu/ . Please cite: https://aclanthology.org/W17-0237/

⸎ To suppress these messages, instantiate ``NLP()`` with ``suppress_banner=True``.


In [49]:
# Execution time is 50 sec on a 2015 Macbook Pro
%time cltk_doc_grc = cltk_nlp_grc.analyze(text=thucydides)

# You will be asked to download some models (from CLTK, fastText, and Stanza)

CPU times: user 25.9 s, sys: 4.69 s, total: 30.6 s
Wall time: 25.8 s


In [50]:
print("`Doc.tokens`:", cltk_doc_grc.tokens[:20])

`Doc.tokens`: ['Θουκυδίδης', 'Ἀθηναῖος', 'ξυνέγραψε', 'τὸν', 'πόλεμον', 'τῶν', 'Πελοποννησίων', 'καὶ', 'Ἀθηναίων', ',', 'ὡς', 'ἐπολέμησαν', 'πρὸς', 'ἀλλήλους', ',', 'ἀρξάμενος', 'εὐθὺς', 'καθισταμένου', 'καὶ', 'ἐλπίσας']


In [51]:
print(cltk_doc_grc.words[4])  # πόλεμον ('war')

Word(index_char_start=None, index_char_stop=None, index_token=4, index_sentence=0, string='πόλεμον', pos=noun, lemma='πόλεμος', stem=None, scansion=None, xpos='n-s---ma-', upos='NOUN', dependency_relation='obj', governor=2, features={Case: [accusative], Gender: [masculine], Number: [singular]}, category={F: [neg], N: [pos], V: [neg]}, stop=False, named_entity=None, syllables=None, phonetic_transcription=None, definition=None)


In [52]:
a_tree_grc = DependencyTree.to_tree(cltk_doc_grc.sentences[0])  #81

In [53]:
pprint(a_tree_grc.get_dependencies())

[nmod(Θουκυδίδης_0/noun, Ἀθηναῖος_1/adjective),
 nsubj(ξυνέγραψε_2/verb, Θουκυδίδης_0/noun),
 det(πόλεμον_4/noun, τὸν_3/determiner),
 det(Πελοποννησίων_6/noun, τῶν_5/determiner),
 cc(Πελοποννησίων_6/noun, καὶ_7/coordinating_conjunction),
 conj(Πελοποννησίων_6/noun, Ἀθηναίων_8/noun),
 nmod(πόλεμον_4/noun, Πελοποννησίων_6/noun),
 obj(ξυνέγραψε_2/verb, πόλεμον_4/noun),
 punct(ὡς_10/subordinating_conjunction, ,_9/punctuation),
 mark(ἐπολέμησαν_11/verb, ὡς_10/subordinating_conjunction),
 case(ἀλλήλους_13/pronoun, πρὸς_12/adposition),
 obl(ἐπολέμησαν_11/verb, ἀλλήλους_13/pronoun),
 advcl(ξυνέγραψε_2/verb, ἐπολέμησαν_11/verb),
 advcl(ξυνέγραψε_2/verb, ἀρξάμενος_15/verb),
 advcl(ξυνέγραψε_2/verb, καθισταμένου_17/verb),
 advcl(ξυνέγραψε_2/verb, τεκμαιρόμενος_28/verb),
 punct(ἀρξάμενος_15/verb, ,_14/punctuation),
 advmod(μέγαν_20/adjective, τε_21/adverb),
 cop(μέγαν_20/adjective, ἔσεσθαι_22/auxiliary),
 cc(μέγαν_20/adjective, καὶ_23/coordinating_conjunction),
 det(προγεγενημένων_26/verb, τῶν_25/

In [54]:
print(cltk_doc_grc.sentences_strings[0])
print("")
print("Translation:", "Thucydides, an Athenian, wrote the history of the war between the Peloponnesians and the Athenians, beginning at the moment that it broke out, and believing that it would be a great war, and more worthy of relation than any that had preceded it. This belief was not without its grounds. The preparations of both the combatants were in every department in the last state of perfection; and he could see the rest of the Hellenic race taking sides in the quarrel; those who delayed doing so at once having it in contemplation.")
print("")
a_tree_grc.print_tree()

Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν Πελοποννησίων καὶ Ἀθηναίων , ὡς ἐπολέμησαν πρὸς ἀλλήλους , ἀρξάμενος εὐθὺς καθισταμένου καὶ ἐλπίσας μέγαν τε ἔσεσθαι καὶ ἀξιολογώτατον τῶν προγεγενημένων , τεκμαιρόμενος ὅτι ἀκμάζοντές τε ᾖσαν ἐς αὐτὸν ἀμφότεροι παρασκευῇ τῇ πάσῃ καὶ τὸ ἄλλο Ἑλληνικὸν ὁρῶν ξυνιστάμενον πρὸς ἑκατέρους , τὸ μὲν εὐθύς , τὸ δὲ καὶ διανοούμενον .

Translation: Thucydides, an Athenian, wrote the history of the war between the Peloponnesians and the Athenians, beginning at the moment that it broke out, and believing that it would be a great war, and more worthy of relation than any that had preceded it. This belief was not without its grounds. The preparations of both the combatants were in every department in the last state of perfection; and he could see the rest of the Hellenic race taking sides in the quarrel; those who delayed doing so at once having it in contemplation.

root | ξυνέγραψε_2/verb
    └─ nsubj | Θουκυδίδης_0/noun
        └─ nmod | Ἀθηναῖος_1/ad

In [55]:
feature_names_grc, list_of_list_features_grc = cltk_doc_to_features_table(cltk_doc=cltk_doc_grc)

In [56]:
print(feature_names_grc)

['string', 'lemma', 'embedding', 'is_stop', 'lemma', 'pos', 'case', 'gender', 'animacy', 'number', 'definiteness', 'degree', 'strength', 'verbform', 'tense', 'mood', 'aspect', 'voice', 'person', 'polarity', 'politeness', 'clusivity', 'evidentiality', 'strength', 'governing_word', 'governor_case', 'governor_gender', 'governor_animacy', 'governor_number', 'governor_definiteness', 'governor_degree', 'governor_strength', 'governor_verbform', 'governor_tense', 'governor_mood', 'governor_aspect', 'governor_voice', 'governor_person', 'governor_polarity', 'governor_politeness', 'governor_clusivity', 'governor_evidentiality', 'governor_strength', 'governing_relationship']


In [57]:
print("len() of feature instances (one for each token):", len(list_of_list_features_grc))
print("")
print("Example of one instance row:", list_of_list_features_grc[4])

len() of feature instances (one for each token): 11517

Example of one instance row: ['πόλεμον', 'πόλεμος', array([ 0.026979, -0.201583,  0.403359, -0.283483, -0.009569, -0.199355,
        0.045749,  0.004704,  0.142157,  0.256203,  0.597056,  0.051038,
       -0.163049, -0.373365,  0.036745, -0.091862,  0.297455,  0.215132,
        0.038652, -0.162991, -0.365283, -0.290551,  0.702512, -0.229625,
        0.36157 , -0.160029,  0.142917, -0.015726,  0.27188 ,  0.122487,
        0.138659,  0.046546,  0.257647,  0.176612, -0.379691, -0.106146,
       -0.111118, -0.351822, -0.149012,  0.266751,  0.07048 ,  0.036137,
       -0.087169, -0.609234,  0.015767,  0.18534 ,  0.090315, -0.163389,
        0.080256, -0.213097, -0.208633, -0.153822, -0.126109,  0.170786,
        0.364725, -0.21801 ,  0.445456, -0.046112,  0.442441, -0.444369,
        0.045849, -0.107836,  0.046578, -0.171076,  0.377415,  0.554179,
        0.566092, -0.06977 ,  0.598988,  0.273217, -0.489505,  0.067709,
       -0.388921

In [58]:
# Putting these together for easier reading
pprint(list(zip(feature_names_grc, list_of_list_features_grc[4])))

[('string', 'πόλεμον'),
 ('lemma', 'πόλεμος'),
 ('embedding',
  array([ 0.026979, -0.201583,  0.403359, -0.283483, -0.009569, -0.199355,
        0.045749,  0.004704,  0.142157,  0.256203,  0.597056,  0.051038,
       -0.163049, -0.373365,  0.036745, -0.091862,  0.297455,  0.215132,
        0.038652, -0.162991, -0.365283, -0.290551,  0.702512, -0.229625,
        0.36157 , -0.160029,  0.142917, -0.015726,  0.27188 ,  0.122487,
        0.138659,  0.046546,  0.257647,  0.176612, -0.379691, -0.106146,
       -0.111118, -0.351822, -0.149012,  0.266751,  0.07048 ,  0.036137,
       -0.087169, -0.609234,  0.015767,  0.18534 ,  0.090315, -0.163389,
        0.080256, -0.213097, -0.208633, -0.153822, -0.126109,  0.170786,
        0.364725, -0.21801 ,  0.445456, -0.046112,  0.442441, -0.444369,
        0.045849, -0.107836,  0.046578, -0.171076,  0.377415,  0.554179,
        0.566092, -0.06977 ,  0.598988,  0.273217, -0.489505,  0.067709,
       -0.388921,  0.130601, -0.258088,  0.148842, -0.084431